# day 6

https://adventofcode.com/2018/day/6

In [ ]:
import os

import eri.logging as logging

In [ ]:
FNAME = os.path.join('data', 'day6.txt')

LOGGER = logging.getLogger('day6')
logging.configure()

## part 1

### problem statement:

> The device on your wrist beeps several times, and once again you feel like you're falling.
> 
> "Situation critical," the device announces. "Destination indeterminate. Chronal interference detected. Please specify new target coordinates."
> 
> The device then produces a list of coordinates (your puzzle input). Are they places it thinks are safe or dangerous? It recommends you check manual page 729. The Elves did not give you a manual.
> 
> If they're dangerous, maybe you can minimize the danger by finding the coordinate that gives the largest distance from the other points.
> 
> Using only the Manhattan distance, determine the area around each coordinate by counting the number of integer X,Y locations that are closest to that coordinate (and aren't tied in distance to any other coordinate).
> 
> Your goal is to find the size of the largest area that isn't infinite. For example, consider the following list of coordinates:
> 
>     1, 1
>     1, 6
>     8, 3
>     3, 4
>     5, 5
>     8, 9
> 
> If we name these coordinates A through F, we can draw them on a grid, putting 0,0 at the top left:
> 
>     ..........
>     .A........
>     ..........
>     ........C.
>     ...D......
>     .....E....
>     .B........
>     ..........
>     ..........
>     ........F.
>
> This view is partial - the actual grid extends infinitely in all directions. Using the Manhattan distance, each location's closest coordinate can be determined, shown here in lowercase:
> 
>     aaaaa.cccc
>     aAaaa.cccc
>     aaaddecccc
>     aadddeccCc
>     ..dDdeeccc
>     bb.deEeecc
>     bBb.eeee..
>     bbb.eeefff
>     bbb.eeffff
>     bbb.ffffFf
>
> Locations shown as . are equally far from two or more coordinates, and so they don't count as being closest to any.
> 
> In this example, the areas of coordinates A, B, C, and F are infinite - while not shown here, their areas extend forever outside the visible grid. However, the areas of coordinates D and E are finite: D is closest to 9 locations, and E is closest to 17 (both including the coordinate's location itself). Therefore, in this example, the size of the largest area is 17.
> 
> What is the size of the largest area that isn't infinite?

#### loading data

In [ ]:
import pandas as pd
logging.getLogger('matplotlib').setLevel(logging.WARNING)

def load_data(fname=FNAME):
    with open(fname, 'r') as f:
        return pd.DataFrame(
            [[int(_) for _ in row.strip().split(', ')] for row in f],
            columns=['x', 'y']
        )

In [ ]:
load_data().head()

#### function def

In [ ]:
grid = q_1(test_data)

In [ ]:
row = grid.iloc[0]

In [ ]:
row

In [ ]:
(coordinates - row).abs().sum(axis=1)

In [ ]:
import tqdm

In [ ]:
def q_1(coordinates):
    xmin = coordinates.x.min() - 1
    ymin = coordinates.y.min() - 1
    xmax = coordinates.x.max() + 1
    ymax = coordinates.y.max() + 1
    
    grid = pd.DataFrame([{'x': x, 'y': y} for x in range(xmin, xmax + 1) for y in range(ymin, ymax + 1)])
    
    def dist_func(row):
        dists = (coordinates - row).abs().sum(axis=1)
        return {
            'nearest_coord': dists.idxmin(),
            'is_tie': (dists == dists.min()).sum() > 1,
        }
    
    grid = grid.join(grid.progress_apply(dist_func, axis=1, result_type='expand'))
    
    grid.loc[:, 'is_edge'] = (
        (grid.x == xmin)
        | (grid.x == xmax)
        | (grid.y == ymin)
        | (grid.y == ymax)
    )
    
    coord_sum = grid.groupby('nearest_coord') \
        .agg({'nearest_coord': 'count', 'is_edge': 'max'}) \
        .rename(columns={'nearest_coord': 'ct', 'is_edge': 'is_inf'})
    
    return grid, coord_sum, coord_sum[~coord_sum.is_inf].ct.max()
    return coord_sum[~coord_sum.is_inf].ct.max()

#### tests

In [ ]:
test_data = pd.DataFrame([
    [1, 1],
    [1, 6],
    [8, 3],
    [3, 4],
    [5, 5],
    [8, 9]
], columns=['x', 'y']
)

def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 17
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
grid, coord_sum, x = q_1(load_data())

In [ ]:
grid.head()

In [ ]:
coord_sum.sort_values(by='ct')

In [ ]:
x

stolen from [here](https://www.reddit.com/r/adventofcode/comments/a3kr4r/2018_day_6_solutions/):

In [ ]:
import numpy as np
from scipy.spatial import distance

# read the data using scipy
points = load_data()

# build a grid of the appropriate size - note the -1 and +2 to ensure all points
# are within the grid
xmin, ymin = points.min(axis=0) - 1
xmax, ymax = points.max(axis=0) + 2

# and use mesgrid to build the target coordinates
xgrid, ygrid = np.meshgrid(np.arange(xmin, xmax), np.arange(xmin, xmax))
targets = np.dstack([xgrid, ygrid]).reshape(-1, 2)

# happily scipy.spatial.distance has cityblock (or manhatten) distance out
# of the box
cityblock = distance.cdist(points, targets, metric='cityblock')
# the resulting array is an input points x target points array
# so get the index of the maximum along axis 0 to tie each target coordinate
# to closest ID
closest_origin = np.argmin(cityblock, axis=0)
# we need to filter out points with competing closest IDs though
min_distances = np.min(cityblock, axis=0)
competing_locations_filter = (cityblock == min_distances).sum(axis=0) > 1
# note, integers in numpy don't support NaN, so make the ID higher than
# the possible point ID
closest_origin[competing_locations_filter] = len(points) + 1
# and those points around the edge of the region for "infinite" regions
closest_origin = closest_origin.reshape(xgrid.shape)
infinite_ids = np.unique(np.vstack([
    closest_origin[0],
    closest_origin[-1],
    closest_origin[:, 0],
    closest_origin[:, -1]
]))
closest_origin[np.isin(closest_origin, infinite_ids)] = len(points) + 1

# and because we know the id of the "null" data is guaranteed to be last
# in the array (it's highest) we can index it out before getting the max
# region size
print(np.max(np.bincount(closest_origin.ravel())[:-1]))

## part 2

### problem statement:

> On the other hand, if the coordinates are safe, maybe the best you can do is try to find a region near as many coordinates as possible.
> 
> For example, suppose you want the sum of the Manhattan distance to all of the coordinates to be less than 32. For each location, add up the distances to all of the given coordinates; if the total of those distances is less than 32, that location is within the desired region. Using the same coordinates as above, the resulting region looks like this:
> 
>     ..........
>     .A........
>     ..........
>     ...###..C.
>     ..#D###...
>     ..###E#...
>     .B.###....
>     ..........
>     ..........
>     ........F.
>
> In particular, consider the highlighted location 4,3 located at the top middle of the region. Its calculation is as follows, where abs() is the absolute value function:
> 
>     Distance to coordinate A: abs(4-1) + abs(3-1) =  5
>     Distance to coordinate B: abs(4-1) + abs(3-6) =  6
>     Distance to coordinate C: abs(4-8) + abs(3-3) =  4
>     Distance to coordinate D: abs(4-3) + abs(3-4) =  2
>     Distance to coordinate E: abs(4-5) + abs(3-5) =  3
>     Distance to coordinate F: abs(4-8) + abs(3-9) = 10
>     Total distance: 5 + 6 + 4 + 2 + 3 + 10 = 30
> 
> Because the total distance to all coordinates (30) is less than 32, the location is within the region.
> 
> This region, which also includes coordinates D and E, has a total size of 16.
> 
> Your actual region will need to be much larger than this example, though, instead including all locations with a total distance of less than 10000.
> 
> What is the size of the region containing all locations which have a total distance to all given coordinates of less than 10000?

#### function def

In [ ]:
def q_2(records):
    records = parse(records)
    
    sleep_summary = []
    for k, d in records.items():
        sleep_history = sum(d.values())
        sleep_time = sleep_history.argmax()
        sleep_time_time = sleep_history.max()
        sleep_summary.append((k, int(k) * sleep_time, sleep_time_time))
    
    return max(sleep_summary, key=lambda rec: rec[2])[1]

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 4455
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
LOGGER.setLevel(logging.INFO)
q_2(load_data())

In [ ]:
xmin, ymin = points.min(axis=0) - 1
xmax, ymax = points.max(axis=0) + 2

# and use mesgrid to build the target coordinates
xgrid, ygrid = np.meshgrid(np.arange(xmin, xmax), np.arange(xmin, xmax))
targets = np.dstack([xgrid, ygrid]).reshape(-1, 2)

# happily scipy.spatial.distance has cityblock (or manhatten) distance out
# of the box
cityblock = distance.cdist(points, targets, metric='cityblock')

# turns out using this method the solution is easier that before - simply
# sum the distances for each possible grid location
origin_distances = cityblock.sum(axis=0)
# set the value of appropriate distances to 1, with the remainder as zero
region = np.where(origin_distances < 10000, 1, 0)
# and the sum is the result.
print(region.sum())

fin